The code in this notebook was run on the Databricks notebook, following on from the code found in the dataframe cleaning notebook

In [ ]:
# import for performing window functions
from pyspark.sql.window import Window
# join df_pin and df_geo dataframes on index
pin_geo = df_pin.join(df_geo, df_pin.ind == df_geo.ind)
# join df_pin and df_user and create temp view for SQL query
df_pin.join(df_user, df_pin.ind == df_user.ind).createOrReplaceTempView("category_age")
# SQL query to create age group column
pin_user_age_group = spark.sql(
    "SELECT CASE \
        WHEN age between 18 and 24 then '18-24' \
        WHEN age between 25 and 35 then '25-35' \
        WHEN age between 36 and 50 then '36-50' \
        WHEN age > 50 then '50+' \
        END as age_group, * FROM category_age")

## Find the most popular category in each country

In [ ]:
# create partition by country and order by category_count descending
windowCountryByCatCount = Window.partitionBy("country").orderBy(col("category_count").desc())
# find the most popular category in each country
pin_geo.groupBy("country", "category") \
.agg(count("category") \
.alias("category_count")) \
.withColumn("rank", row_number().over(windowCountryByCatCount)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 9|
 Albania| art| 14|
 Algeria| quotes| 19|
 American Samoa| tattoos| 8|
 Andorra| tattoos| 7|
 Angola| education| 4|
 Anguilla|diy-and-crafts| 4|
Antarctica (the t...| tattoos| 4|
 Antigua and Barbuda| art| 6|
 Argentina| tattoos| 12|
 Armenia| art| 4|
 Aruba| art| 7|
 Australia| christmas| 5|
 Austria| tattoos| 6|
 Azerbaijan|diy-and-crafts| 5|
 Bahamas| art| 9|
 Bahrain| travel| 5|
 Bangladesh| christmas| 5|
 Barbados| travel| 2|
 Belarus| travel| 2|
+--------------------+--------------+--------------+
only showing top 20 rows

## Find the most popular category each year

In [ ]:
# create partition by year and order by category_count descending
windowYearByCatCount = Window.partitionBy("post_year").orderBy(col("category_count").desc())
# find which was the most popular category each year between 2018 and 2022
pin_geo.withColumn("post_year", year("timestamp")) \
.filter(col("post_year") >= 2018) \
.filter(col("post_year") <= 2022) \
.groupBy("post_year", "category") \
.agg(count("category").alias("category_count")) \
.withColumn("rank", row_number().over(windowYearByCatCount)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()

+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018|diy-and-crafts| 25|
 2019|diy-and-crafts| 19|
 2020| mens-fashion| 22|
 2021| art| 33|
 2022| christmas| 27|
+---------+--------------+--------------+

## Find the user with the most followers in each country
### Based on the above query, find the country with the user with most followers

In [ ]:
# create partition by country and order by follower_count descending
windowCountryByFollowers = Window.partitionBy("country").orderBy(col("follower_count").desc())

# find the user with the most followers in each country
max_followers_by_country = \
    df_pin.join(df_geo, df_pin.ind == df_geo.ind) \
    .withColumn("rank", row_number().over(windowCountryByFollowers)) \
    .filter(col("rank") == 1) \
    .select("country", "poster_name", "follower_count")

# get highest number of followers from all countries
max_followers_all_countries = max_followers_by_country.select(max("follower_count")).collect()[0][0]

# find the country with the user with most followers
country_with_max_followers = \
    max_followers_by_country \
    .select("*") \
    .where(col("follower_count") == max_followers_all_countries)

max_followers_by_country.show()
country_with_max_followers.show()

+--------------------+--------------------+--------------+
 country| poster_name|follower_count|
+--------------------+--------------------+--------------+
 Afghanistan| 9GAG| 3000000|
 Albania| The Minds Journal| 5000000|
 Algeria| YourTango| 942000|
 American Samoa| Mamas Uncut| 8000000|
 Andorra|Teachers Pay Teac...| 1000000|
 Angola| Tastemade| 8000000|
 Anguilla|Kristen | Lifesty...| 92000|
Antarctica (the t...| Refinery29| 1000000|
 Antigua and Barbuda|Country Living Ma...| 1000000|
 Argentina| Cheezburger| 2000000|
 Armenia|Michelle {CraftyM...| 892000|
 Aruba| GQ Magazine| 874000|
 Australia| Mujer de 10 | 2000000|
 Austria| Diply| 251000|
 Azerbaijan| Style Me Pretty| 6000000|
 Bahamas| Handmade Charlotte| 948000|
 Bahrain|Alea Milham | Pre...| 332000|
 Bangladesh|Better Homes and ...| 4000000|
 Barbados|The Creativity Ex...| 410000|
 Belarus| Something Turquoise| 312000|
+--------------------+--------------------+--------------+
only showing top 20 rows

+--------------+-----------+--------------+
 country|poster_name|follower_count|
+--------------+-----------+--------------+
American Samoa|Mamas Uncut| 8000000|
 Angola| Tastemade| 8000000|
+--------------+-----------+--------------+

## Find the most popular category for different age groups

In [ ]:
# create partition by age_group and order by category_count descending
windowAgeGroup = Window.partitionBy("age_group").orderBy(col("category_count").desc())
# find the most popular category for different age groups
pin_user_age_group.groupBy("age_group", "category") \
.agg(count("category").alias("category_count")) \
.withColumn("rank", row_number().over(windowAgeGroup)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()

+---------+------------+--------------+
age_group| category|category_count|
+---------+------------+--------------+
 18-24| art| 68|
 25-35| travel| 31|
 36-50| travel| 28|
 50+|mens-fashion| 10|
+---------+------------+--------------+

## Find the median follower count for different age groups

In [ ]:
# find the median follower count for different age groups
pin_user_age_group \
.select("user_name", "date_joined", "age_group", "follower_count") \
.distinct() \
.groupBy("age_group") \
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("age_group") \
.show()

+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 18-24| 56000|
 25-35| 19000|
 36-50| 6000|
 50+| 1000|
+---------+---------------------+

## Find out how many users joined each year

In [ ]:
# find out how many users joined each year
df_user.withColumn("post_year", year("date_joined")) \
.drop("ind") \
.distinct() \
.groupBy("post_year") \
.agg(count("user_name").alias("number_users_joined")) \
.orderBy("post_year") \
.show()

+---------+-------------------+
post_year|number_users_joined|
+---------+-------------------+
 2015| 215|
 2016| 398|
 2017| 153|
+---------+-------------------+

## Find the median follower count of users based on their joining year

In [ ]:
# find the median follower count of users based on their joining year
pin_user_age_group \
.select("user_name", "date_joined", "follower_count") \
.distinct() \
.withColumn("post_year", year("date_joined")) \
.groupBy("post_year") \
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("post_year") \
.show()

+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2015| 60000|
 2016| 14000|
 2017| 2000|
+---------+---------------------+

## Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of

In [ ]:
# find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of
pin_user_age_group \
.select("user_name", "age_group", "date_joined", "follower_count") \
.distinct() \
.withColumn("post_year", year("date_joined")) \
.groupBy("post_year", "age_group") \
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("post_year", "age_group") \
.show()

+---------+---------+---------------------+
post_year|age_group|median_follower_count|
+---------+---------+---------------------+
 2015| 18-24| 115000|
 2015| 25-35| 42000|
 2015| 36-50| 8000|
 2015| 50+| 1000|
 2016| 18-24| 40000|
 2016| 25-35| 17000|
 2016| 36-50| 8000|
 2016| 50+| 2000|
 2017| 18-24| 11000|
 2017| 25-35| 1000|
 2017| 36-50| 3000|
 2017| 50+| 760|
+---------+---------+---------------------+